Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [103]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from random import randint
from copy import deepcopy, copy
from typing import Callable
import math


## The *Nim* and *Nimply* classes

In [104]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [105]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [106]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [107]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [108]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [109]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply




Copyright **(c)** 2023 Abdelouahab Moubane <abdelmub@gmail.com>
https://github.com/AbdelouahabMoubane

In [110]:
def agent_rulebased(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 1]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply

In [111]:
#not good
def ES(p):  
    def My_strategy(state: Nim) -> Nimply:
        n_rows = 5
        xor_candidates = list()
        #p = list(np.ones(2**n_rows-1))
        for i in range(2**(n_rows-1)):
            xor_candidates.append(i)
        analysis = analize(state)
        logging.debug(f"analysis:\n{pformat(analysis)}")
        spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == xor_candidates[p.argmax()]] 
        #random.choices(xor_candidates,p,k=1)[0] #xor_candidates[p.argmax()]
        if not spicy_moves:
            spicy_moves = list(analysis["possible_moves"].keys())
        ply = random.choice(spicy_moves)    
        return ply
    return My_strategy



In [112]:
# very good
def ES1(p): 
    
    def My_strategy(state: Nim) -> Nimply:
        temp = copy(p) 
        analysis = analize(state)
        logging.debug(f"analysis:\n{pformat(analysis)}")
        spicy_moves = []
        while len(spicy_moves) == 0:
            spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == temp.argmax()] 
            #random.choices(xor_candidates,p,k=1)[0] #xor_candidates[p.argmax()]
            
            if len(spicy_moves)==0:
                temp = np.delete(temp, temp.argmax()) 
            else:
                break    
    
        ply = random.choice(spicy_moves) 
        
        return ply
    return My_strategy

## Oversimplified match

In [113]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 4 7 9>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=7)
INFO:root:status: <1 3 4 0 9>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=4)
INFO:root:status: <1 3 0 0 9>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 0 0 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=9)
INFO:root:status: <0 3 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=2)
INFO:root:status: <0 1 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 0 0 0 0>
INFO:root:status: Player 1 won!


## Fitness function

In [114]:
def Comparison(n_rows, n_games, strategy0, strategy1):
    count = 0
    for i in range(n_games):

        logging.getLogger().setLevel(logging.INFO)

        strategy = (strategy0, strategy1)

        nim = Nim(n_rows)
        #logging.info(f"init : {nim}")
        #player=randint(0,1)
        if i < n_games/2:
            player = 0
        else:
            player = 1    
        while nim:
            ply = strategy[player](nim)
            #logging.info(f"ply: player {player} plays {ply}")
            nim.nimming(ply)
            #logging.info(f"status: {nim}")
            player = 1 - player
            #logging.info(f"status: Player {player} won!")
        if player == 0:
            count+=1   
    return count/n_games

fitness = Comparison        

## Parameters game Nim

In [115]:
n_rows = 5
n_games = 20

In [116]:
Comparison(n_rows, n_games, agent_rulebased, optimal)

1.0

## Population

In [117]:
mu = 0
sigma = 2
sigma_pop = 2
n = 2**(n_rows-1)

population_size = 100
population = list()
for i in range(population_size):
    # population.append(abs(np.ones(n) + np.random.normal(mu,sigma_pop, n)))
    population.append(np.ones(n) + np.random.exponential(0.2, n))

population[0]

array([1.05666129, 1.15797482, 1.3263528 , 1.07950919, 1.1307822 ,
       1.21459833, 1.23475064, 1.14090486, 1.02814011, 1.02370368,
       1.06913992, 1.07358765, 1.2479187 , 1.12860309, 1.97399026,
       1.00908893])

## Parent selection

## Offspring selection (1+λ)-ES

In [118]:
def offspring(parent, mean, variance, lam, n):
    best_offspring = list()
    best_offspring.append(parent)
    for i in range(lam):    
        par = copy(parent)
        for j in range(n):
            # if parent[j] < 0.5:
            #     par[j] = 0
            # else:
            par[j] = abs(par[j]+np.random.normal(mean, variance, 1))[0]  
            # par[j] = par[j]+np.random.gamma(2,2, 1)  
        #par = abs(par + np.random.normal(mean, variance, len(parent)))

    best_offspring.append(par)

    return best_offspring           



In [119]:
steps = 200
iterations = 0
lam = 20
n = 2**(n_rows-1)
best_parent = random.choice(population)
for i in range(steps):
    if fitness(n_rows, n_games, ES1(best_parent), optimal) > 0.85:
        break
    tot_offspring = offspring(best_parent, mu, sigma, lam, n)
    fit = list()

    for j in range(len(tot_offspring)):
        fit.append(fitness(n_rows, n_games, ES1(tot_offspring[j]), optimal))

    combined = list(zip(tot_offspring, fit))
    combined_sorted = sorted(combined, key=lambda x: x[1], reverse=True)
    top_fit = combined_sorted[:9]
    top_individuals = [elem[0] for elem in top_fit]
        
    best_parent = np.array(random.choice(top_individuals))
    iterations +=1   

best_parent

len(best_parent)

16

In [120]:
best_parent

array([ 4.83384266, 11.02170902,  7.7559786 ,  7.81871358,  0.45796079,
       12.90568604,  3.38263864,  8.31173991,  5.18192479,  6.65010054,
       25.22871268, 11.4342132 , 15.30281033, 11.64000012,  0.56622553,
       11.42411205])

In [127]:
Comparison(n_rows, n_games, ES1(best_parent), optimal)

0.95

In [122]:
iterations

35